## Disclaimer

**Environment:** This notebook has been tested with:

- Julia v1.11.x
- JuMP.jl v1.26.x
- HiGHS v1.18.1

**Author:** Xavier Gandibleux, Nantes (France)


----

## Part 3: Multi-Objective Optimization

Given the bi-objective 01 Unidimensional Knapsack Problem (bi-01UKP) formulated by 
$$z=\max\big\{(p^1x,\ p^2x) \mid wx \le c, \ x\in\{0,1\}^n\big\}$$

and the numerical instance corresponding to 
$$n=5$$
$$p^1=(6, \ 4, \ 4, \ 4, \ 3)$$
$$p^2=(12, 10, \ 5, \ 3, \ 1)$$
$$w=(\ 8, \ 6, \ 4, \ 3, \ 2)$$
$$c=15$$

answer to the following questions:

1. using the package `JuMP`, implement an implicit formulation of the bi-01UKP and display the model.
2. using the package `MultiObjectiveAlgorithms`, compute and display $X_E$ and $Y_N$, a set of efficient solutions and the set of non-dominated points.
3. using the package `Plots`, display $Y_N$ in the objective space $Y$
4. using this learning, implement the required codes to solve instances of the bi-objective General Assignment Problem (biGAP).


## Answers:

----
### 1. using the package `JuMP`, implement an implicit formulation of the bi-01UKP and display the model.

#### declare the package(s) to use:

#### setup the instance to solve

#### setup the formulation

---- 
### 2. using the package `MultiObjectiveAlgorithms`, compute $Y_N$, the set of non-dominated points.

#### setup the MIP solver to use

#### setup the algorithm to use

#### solve the problem

#### query the number of points in $Y_N$

#### query the summary of the 3rd point

#### query the vector values of the 3rd point

#### query the value for the first objective of the 3rd point

#### display $X_E$ and $Y_N$

----
### 3. using the package `Plots`, display $Y_N$ in the objective space $Y$

#### declare the package(s) to use:

#### plot $Y_N$ in $Y$:

----
### 4. using this learning, implement the required codes to solve instances of the bi-objective General Assignment Problem (biGAP).

#### formulation

Consider the following bi-objective generalized assignment (biGAP) problem:

$$\big ( \ \max \sum_{i=1}^{m} \sum_{j=1}^{n} p^1_{ij} x_{ij} \ , \
\max \sum_{i=1}^{m} \sum_{j=1}^{n} p^2_{ij} x_{ij} \ , \
\max \sum_{i=1}^{m} \sum_{j=1}^{n} p^3_{ij} x_{ij} \ \big )$$

$$ s.t \qquad \sum_{j=1}^{n} w_{ij} x_{ij} \le b_i, \quad \forall i \in \{1, \dots, m \}$$

$$ \qquad \qquad \sum_{i=1}^{m} x_{ij} = 1, \quad \forall j \in \{1, \dots, n \}$$


$$x_{ij}\in\{0,1\}, \ \forall i \in \{1, \dots, m \}, \  \forall j \in \{1, \dots, n \}$$

#### generator of instances

Generate an instance $m\times n$ with coefficients randomly generated as follow:
- $1 \le p^1_{ij}, \ p^2_{ij}, \ p^3_{ij}, \ w_{ij} \le 10$ 
- $b_i = \lfloor  \frac{\sum_{j=1}^{m}w_{ij}}{2} \rfloor$ 

#### questions

- generate a numerical instance
- compute $Y_N$, the set of non-dominated points, using HiGHS as MIP solver
- compute $U$, an upper bound set of $Y_N$, using HiGHS as MIP solver
- plot on a same figure $Y_N$ and $U$

In [1]:
m=3; n=10
p1 = rand(1:10,m,n)
p2 = rand(1:10,m,n)
p3 = rand(1:10,m,n)
w  = rand(1:10,m,n)
b  = Vector{Int64}(undef,m)
for i=1:m
    b[i] = floor(Int64,sum(w[i,:])/2.0) 
end

In [2]:
@show p1
@show p2
@show p3
@show w
@show b

p1 = [5 1 10 8 3 5 3 3 7 2; 10 6 1 6 8 3 2 10 6 1; 2 3 1 6 9 7 1 5 4 8]
p2 = [4 6 4 3 1 6 8 2 9 7; 8 8 8 2 4 8 8 1 10 1; 8 7 8 5 9 2 2 7 10 10]
p3 = [4 3 6 4 7 5 9 5 8 4; 8 6 2 2 6 8 5 2 2 3; 2 8 10 3 5 7 5 9 5 5]
w = [5 9 3 5 10 5 7 10 7 8; 4 8 8 6 10 8 10 7 5 1; 10 7 5 8 8 2 8 1 10 3]
b = [34, 33, 31]


3-element Vector{Int64}:
 34
 33
 31

3-GAP with variables in $\{0,1\}$ solved with the epsilon-constraint method using HiGHS: 

In [30]:
using JuMP
using CPLEX
import MultiObjectiveAlgorithms as MOA

GAP = Model()
@variable(GAP, x[1:m, 1:n], Bin)
@expression(GAP, objective1, sum(p1[i,j]*x[i,j] for i = 1:m, j = 1:n) )
@expression(GAP, objective2, sum(p2[i,j]*x[i,j] for i = 1:m, j = 1:n) )
@expression(GAP, objective3, sum(p3[i,j]*x[i,j] for i = 1:m, j = 1:n) )
@objective(GAP, Max, [objective1,objective2,objective3])
@constraint(GAP, [i=1:m], sum(w[i,j]*x[i,j] for j = 1:n) <= b[i])
@constraint(GAP, [j=1:n], sum(x[i,j] for i = 1:m) == 1)

set_optimizer(GAP, () -> MOA.Optimizer(CPLEX.Optimizer))
set_attribute(GAP, MOA.Algorithm(), MOA.KirlikSayin())
#set_attribute(GAP, MOA.Algorithm(), MOA.DominguezRios())
#set_attribute(GAP, MOA.Algorithm(), MOA.TambyVanderpooten())
set_silent(GAP)
optimize!(GAP)

YN = []
for i in 1:result_count(GAP)
    y = round.(Int, objective_value(GAP; result = i))
    push!(YN,y)
    println(i, ": z = ", y)
end

In [31]:
@show result_count(GAP)

result_count(GAP) = 0


0